In [ ]:
pip install dydx-v3-python
pip install dash
pip install dash-core-components
pip install dash-html-components
pip install dash-renderer
pip install plotly

In [1]:
from dydx3 import Client
from dydx3.constants import MARKET_BTC_USD
from dydx3.constants import MARKET_SOL_USD

from dydx3.constants import ORDER_SIDE_SELL
from dydx3.constants import ORDER_TYPE_LIMIT
from dydx3.constants import TIME_IN_FORCE_GTT
from dydx3.constants import POSITION_STATUS_OPEN
from dydx3.constants import ASSET_USDC

from dydx3.constants import OFF_CHAIN_ONBOARDING_ACTION
from dydx3.constants import OFF_CHAIN_KEY_DERIVATION_ACTION
from dydx3.eth_signing import SignOnboardingAction
from dydx3.helpers.requests import request
from dydx3.starkex.helpers import private_key_to_public_key_pair_hex

from web3 import Web3
import asyncio
import websockets
import websocket
import json
import pandas as pd
from datetime import datetime

import random
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import time



## Three modules to deal with DYDX Api: 
- Public module API
- Private module API
- Websocket API
    
### For Private Module API: 
- No Need for ETH Authentication but only API credintials 
    - Get Users Data 
    - Get Open Positions
    - Get Account Data 
    
- Needing ETH Authentication and Private Keys 
    - Creating Orders 
    - Creating withdrawals to an ETH Wallet 
        

In [42]:
# Define a Client 

client = Client(
    host = 'https://api.stage.dydx.exchange', #testnet
    api_key_credentials = {
        'key': '',
        'secret': '',
        'passphrase': ''},
    stark_private_key = "",
    eth_private_key = '',
    default_ethereum_address = '',
    web3=Web3(Web3.HTTPProvider('https://goerli.infura.io/v3/9296b825013449fbb7cdd6f294b33174')))


In [44]:
def user_data():
    user = client.private.get_user()
    return user.data

def all_opsitions(Market=MARKET_BTC_USD):
    all_positions = client.private.get_positions(
      market=Market,  
      status=POSITION_STATUS_OPEN,)
    return all_positions.data

def account_data():
    account_response = client.private.get_account()
    return account_response.data

def place_order():
    placed_order = client.private.create_order(
      position_id= '8902', 
      market=MARKET_SOL_USD,
      side=ORDER_SIDE_SELL,
      order_type=ORDER_TYPE_LIMIT,
      post_only=False,
      size='0.1237',
      price='31140',
      limit_fee='15',
      expiration_epoch_seconds=1689427516,
      time_in_force=TIME_IN_FORCE_GTT)
    return placed_order.data

def withdraw_funds():
    withdrawal = client.private.create_withdrawal(
      position_id=8902, # required for creating the withdrawal signature
      amount='100',
      asset=ASSET_USDC,
        to_address = '0xb3A41dEDC3a55E6Fc7c35381b765b7dda47EbF72',
      expiration_epoch_seconds=1689202336,)
    return withdrawal



### For Public Module API: 
- No Need for ETH Authentication but only API credintials 
    - Get Users Data 
    - Get Open Positions
    - Get Account Data 
    
- Needing ETH Authentication and Private Keys 
    - Creating Orders 
    - Creating withdrawals to an ETH Wallet 
    
Intro to public API: 'https://www.youtube.com/watch?v=xSZMDdIqrQY&ab_channel=Cryptoulette'
        

In [45]:
markets = client.public.get_markets()

market_statistics = client.public.get_stats(
  market=MARKET_BTC_USD, days=1,)

historical_funding = client.public.get_historical_funding(
  market=MARKET_BTC_USD,)

orderbook = client.public.get_orderbook(
  market=MARKET_BTC_USD,)

## For Websockets 

- Connect to your own account which needs signutre and account number
- Connect to all data

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import datetime
import pandas as pd
import numpy as np

trades_list = pd.DataFrame({'time': [datetime.datetime.now()],
                            'side': ['bid'], 'price': [30000], 'size': [0]})
imbalance = []



app = dash.Dash(__name__)
app.layout = html.Div(
    [dcc.Graph(id='real-time-plot'),
        dcc.Interval(
            id='interval-component',
            interval=1 * 1000,)])  

def transform_message(message): 
    global trades_list
    
    if message['type'] == 'subscribed': 
        for side, data in message['contents'].items(): 
            for trade in data: 
                if float(trade['price']) > 35000:
                    pass 
                else: 
                    df = pd.DataFrame({'time': [datetime.datetime.now()],
                                        'side':[side], 'price':[float(trade['price'])], 'size':[float(trade['size'])]})
                    trades_list = pd.concat([trades_list, df])

    if message['type'] == 'channel_data':
        for side, data in message['contents'].items(): 
            if side == 'offset': 
                offset = message['contents']['offset']
            else: 
                for trade in data:
                     if float(trade[0]) > 35000:
                        pass 
                     else: 
                        if trade[1] == 0 : 
                            pass
                        else: 
                            df = pd.DataFrame({'time': [datetime.datetime.now()],
                                                'side': [side], 'price':[float(trade[0])], 'size':[float(trade[1])]})
                            trades_list = pd.concat([trades_list, df])

def run_script():
        def on_open(ws):
            print('Opened')
            channel_data = {"type":"subscribe",
                            'channel': 'v3_orderbook',
                            'id': 'BTC-USD',
                            'includeOffsets': 'true',}
            ws.send(json.dumps(channel_data))
        
        def on_message(ws, message):
            try:
                message = json.loads(message)
                data = transform_message(message)

            except Exception as e: 
                print(e)
            
        def on_close(ws):
            print("closed")
        
        socket = 'wss://api.dydx.exchange/v3/ws'
        ws = websocket.WebSocketApp(socket,
                                   on_open = on_open,
                                   on_message = on_message,
                                   on_close = on_close
                                   )
        ws.run_forever()
        
@app.callback(Output('real-time-plot', 'figure'),
              [Input('interval-component', 'n_intervals')])


def update_graph(n):
    global trades_list, imbalance
    
    aggregated_df = trades_list.groupby(['time', 'price', 'side']).mean(
        'size').reset_index()
    
    # Orderbook Imbalance 
    Order_Book_Imbalance_Percentage = ((aggregated_df['size'].loc[aggregated_df.side == 'bids'].sum()-
                                       aggregated_df['size'].loc[aggregated_df.side == 'asks'].sum())/ 
                                       (aggregated_df['size'].loc[aggregated_df.side == 'bids'].sum()+
                                       aggregated_df['size'].loc[aggregated_df.side == 'asks'].sum()))
    try:
        imbalance.append([Order_Book_Imbalance_Percentage, aggregated_df['price'].iloc[-1]])
    except Exception as e: 
        print(e)
        
    print(len(imbalance))
    
    
# Create the plot
    fig = go.Figure(go.Heatmap(
        x=aggregated_df['time'], y=aggregated_df['price'],
        z=np.nan_to_num(aggregated_df['size']), colorscale='blackbody'))
    fig.update_layout(plot_bgcolor='black',
                      title='Real-Time Scatter Plot',
                      xaxis_title='Time (seconds since epoch)',
                      yaxis_title='Random Integer',
                     yaxis_tickformat=',',yaxis_range=[29000,31000] )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
    run_script()



/var/folders/cj/y2g53yyx4bs9qmksf0q92ww40000gn/T/ipykernel_42044/3389413776.py:90: RuntimeWarning:

invalid value encountered in scalar divide



1
Opened
